In [15]:
from ucsc_genomes_downloader import Genome
import pandas as pd
import numpy as np
from retrieve_preprocessed_dataset import *

In [4]:
epigenomes, labels = retrieve_epigenomes_labels()

In [5]:
def to_bed(data:pd.DataFrame)->pd.DataFrame:
    """Return bed coordinates from given dataset."""
    return data.reset_index()[data.index.names]

In [18]:
from keras_bed_sequence import BedSequence

def one_hot_encode(genome:Genome, data:pd.DataFrame, nucleotides:str="actg")->np.ndarray:
    return np.array(BedSequence(
        genome,
        bed=to_bed(data),
        nucleotides=nucleotides,
        batch_size=1
    ))

In [19]:
def flat_one_hot_encode(genome:Genome, data:pd.DataFrame, window_size:int, nucleotides:str="actg")->np.ndarray:
    return one_hot_encode(genome, data, nucleotides).reshape(-1, window_size*4).astype(int)

In [20]:
def to_dataframe(x:np.ndarray, window_size:int, nucleotides:str="actg")->pd.DataFrame:
    return pd.DataFrame(
        x,
        columns = [
            f"{i}{nucleotide}"
            for i in range(window_size)
            for nucleotide in nucleotides
        ]
    )

In [12]:
def generate_sequences():

    window_size = 256 # nucleotidi

    genome = Genome('hg38')


    sequences = {
        region: to_dataframe(
            flat_one_hot_encode(genome, data, window_size),
            window_size
        )
        for region, data in epigenomes.items()
    }
    
    sequences["promoters"].to_csv('./pre_processed_dataset/sequence_promoters.csv') 
    sequences['enhancers'].to_csv('./pre_processed_dataset/sequence_enhancers.csv')  

In [13]:
generate_sequences()

Loading chromosomes for genome hg38:   0%|                                                      | 0/25 [00:00<…